# Estimator extraction

### Content

+ [1. Notebook description](#1.-Notebook-Description)
+ [2. Extraction](#2.-Extraction)
    + [2.1 LDA](#2.1-LDA)
    + [2.2 SVM](#2.2-SVM)
    + [2.3 CSP](#2.3-CSP)

---

# 1. Notebook Description

With the gridsearch results aggregated to numpy binary files for each model run, we now want to find the estimator meta parameters for each group that yielded the best score. We know which models were the best from the plot notebooks, but it's not yet clear which estimator params resulted in those scores.
Each model, good or bad, will find the best parameter set and, with some thresholding, we want to isolate those for each model group (LDA, SVC and CSP).

---

**Imports:**

In [1]:
from digits.utils import dotdict

import numpy as np
import pandas as pd
import glob
from itertools import combinations

---
# 2. Extraction


Given the best model from the plots results, we will first open the numpy binary files containing the result data for that specific model for **each** subject and iteratively only load the `results` dictionary in it to an equally named variable.

The keys in this dictionary are tuples of the form `(i,j)` where `i` and `j` are the digits we tested. Remember, that we have **CV** and **final** results for each digit combination. Because we don't care about the subject ID here we will simply append the current **final** score and the associated parameter(s) to a list.

After reading in all model+sbuject results we create a pandas dataframe from the lists we just filled.

---
## 2.1 LDA

Extract parameters for the best LDA model.
The only variable estimotor here is the regularization parameter `shrinkage`. This can either be set to `auto` (Ledoit-Wolf) or a value in `np.linspace(0,0.2,10)`.

In [2]:
model = 'lda_ss2_201_700_fft_None_200_40bins_nopower'

shrinks = []
scores = []
for resfile in glob.glob('results/'+model+'*.npz'):
    objs = np.load(resfile)
    results = dotdict(objs["results"].reshape(-1)[0])
    data = results.data
    for comb in combinations(np.arange(10), 2):
        for finalscore in data[comb]:
            scores.append(finalscore[0])
            shrink = finalscore[1]['lda__shrinkage']
            shrinks.append(shrink)

df_lda = pd.DataFrame({'score':scores, 'shrinkage':shrinks})

/usr/lib/python3/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/lib/python3/dist-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


After we create the data frame, we will describe the dataset by aggregating all scores (over subjects $\times$ combinations $=1440$) with three functions: `count`, `mean` and `median`. We can optionally threshold all results to only list the most common or best parameters. Because aggregate created a two-level index we'll have to specify the column with a tuple, e.g `('score', 'count')` or `('score', 'mean')`.

In [3]:
dfg = df_lda.groupby(['shrinkage']).agg({'score': ['count', 'mean', 'median', 'std', 'min', 'max']})

In [4]:
threshold = max(dfg[('score', 'count')])/2
#threshold = 0
dftmp = dfg[dfg[('score', 'count')] > threshold]
dftmp.sort_values(by=('score','mean'), ascending=False)

score                                                  
          count      mean    median       std       min       max
shrinkage                                                        
0.044444    168  0.720001  0.725795  0.109397  0.428571  0.947368
0.111111    146  0.718396  0.730769  0.109963  0.416667  0.964286
0.066667    156  0.714293  0.717955  0.101758  0.370370  0.931298
0.088889    156  0.713823  0.708644  0.100652  0.444444  0.961538
0.177778    143  0.711518  0.714286  0.099387  0.428571  1.000000
0.155556    116  0.705833  0.692308  0.103993  0.384615  0.928571
0.200000    195  0.703063  0.713178  0.103538  0.423077  0.962963
0.133333    135  0.697144  0.699187  0.100179  0.448276  0.892857
0.022222    141  0.658584  0.653846  0.114928  0.304348  0.961538

In [5]:
print(_.to_latex())

\begin{tabular}{lrrrrrr}
\toprule
{} & score &           &           &           &           &           \\
{} & count &      mean &    median &       std &       min &       max \\
shrinkage &       &           &           &           &           &           \\
\midrule
0.044444  &   168 &  0.720001 &  0.725795 &  0.109397 &  0.428571 &  0.947368 \\
0.111111  &   146 &  0.718396 &  0.730769 &  0.109963 &  0.416667 &  0.964286 \\
0.066667  &   156 &  0.714293 &  0.717955 &  0.101758 &  0.370370 &  0.931298 \\
0.088889  &   156 &  0.713823 &  0.708644 &  0.100652 &  0.444444 &  0.961538 \\
0.177778  &   143 &  0.711518 &  0.714286 &  0.099387 &  0.428571 &  1.000000 \\
0.155556  &   116 &  0.705833 &  0.692308 &  0.103993 &  0.384615 &  0.928571 \\
0.200000  &   195 &  0.703063 &  0.713178 &  0.103538 &  0.423077 &  0.962963 \\
0.133333  &   135 &  0.697144 &  0.699187 &  0.100179 &  0.448276 &  0.892857 \\
0.022222  &   141 &  0.658584 &  0.653846 &  0.114928 &  0.304348 &  0.961538 \\

---
## 2.2 SVM

Similar to the [LDA](#2.1-LDA) above we load all subject specific results from a named model (the best).
This time we're going to create two data frames, because we ran two distinct grid searches in accordance with the method described by Keerthi & Lin 2003.

The resulting scores are 2-tuples, with the first element holding scores and values for the linear SVM and the second element holding the score for the subsequent non-linear SVM based on the best C from the first step.

First we'll extract only the linear scores.

In [6]:
model = 'svc_ss2_201_700_fft_None_200_40bins_nopower'

cs = []
scores = []
for resfile in glob.glob('results/'+model+'*.npz'):
    objs = np.load(resfile)
    config = dotdict(objs["config"].reshape(-1)[0])
    results = dotdict(objs["results"].reshape(-1)[0])
    data = results.data
    for comb in combinations(np.arange(10), 2):
        (score, _) = data[comb]
        scores.append(score[0])
        cs.append(score[1]['svc__C'])

df_lin = pd.DataFrame({'score':scores, 'C':cs})

Then, with the exact same loop as above we will take the second element in the result tuple to save the score and $\gamma,C$ values for the rbf SVM.

In [7]:
scores = []
gammas = []
cs = []
for resfile in glob.glob('results/'+model+'*.npz'):
    objs = np.load(resfile)
    config = dotdict(objs["config"].reshape(-1)[0])
    results = dotdict(objs["results"].reshape(-1)[0])
    data = results.data
    for comb in combinations(np.arange(10), 2):
        (_, score) = data[comb]
        scores.append(score[0])
        gammas.append(score[1]['svc__gamma'])
        cs.append(score[1]['svc__C'])

df_rbf = pd.DataFrame({'score':scores, 'gamma':gammas, 'C':cs})

Now we can look at each data frame `df_lin` and `df_rbf` to identify the best estimators.
First, for the linear SVM, thresholded for estimators which won at least 10 times.

In [8]:
dfg = df_lin.groupby(['C']).agg({'score': ['count', 'mean', 'median', 'std', 'min', 'max']})

In [9]:
threshold = max(dfg[('score', 'count')])/2
#threshold = 0
dftmp = dfg[dfg[('score', 'count')] > threshold]
dftmp.sort_values(by=('score','mean'), ascending=False)

score                                                  
             count      mean    median       std       min       max
C                                                                   
1.274275e-06   526  0.711809  0.720000  0.098427  0.321429  0.947368
4.281332e-06   294  0.702700  0.707505  0.105202  0.360000  0.925926
3.792690e-07   344  0.699200  0.702997  0.104460  0.407407  0.941176

In [25]:
#print(dftmp.sort_values(by=('score','mean'), ascending=False).to_latex())

Second, we'll look at the best RBF parameters, also thresholding by estimators that won at least 10 times.
This is the first time that thresholding is really handy, because there are potentially a lot of $\gamma$-$C$ combinations.

In [12]:
dfg = df_rbf.groupby(['gamma','C']).agg({'score': ['count', 'mean', 'median', 'std', 'min', 'max']})

In [13]:
threshold = max(dfg[('score', 'count')])/2
#threshold = 0
dftmp = dfg[dfg[('score', 'count')] > threshold]
dftmp.sort_values(by=('score','mean'), ascending=False)

score                                          \
                        count      mean    median       std       min   
gamma        C                                                          
5.000000e-09 37.926902    313  0.689934  0.694915  0.108590  0.407407   
             127.427499   503  0.689840  0.689655  0.092497  0.458333   
             428.133240   275  0.678704  0.682171  0.105359  0.321429   

                                   
                              max  
gamma        C                     
5.000000e-09 37.926902   0.965517  
             127.427499  0.962963  
             428.133240  0.931298

In [24]:
#print(dftmp.sort_values(by=('score','mean'), ascending=False).to_latex())

---
## 2.3 CSP

In [16]:
model = 'cspflatlda_ss4_201_700_fft_None_80_40bins_nopower'

comps = []; scores = []; shrinks = []

for resfile in glob.glob('results/'+model+'*.npz'):
    objs = np.load(resfile)
    config = dotdict(objs["config"].reshape(-1)[0])
    results = dotdict(objs["results"].reshape(-1)[0])
    data = results.data
    for comb in combinations(np.arange(10), 2):
        score = data[comb][0]
        scores.append(score[0])
        comps.append(score[1]['csp__n_components'])
        shrinks.append(score[1]['lda__shrinkage'])

dfcsp = pd.DataFrame({'score':scores, 'components':comps, 'shrinkage':shrinks})

In [17]:
dfg = dfcsp.groupby(['shrinkage','components']).agg({'score': ['count', 'mean', 'median', 'std', 'min', 'max'] })

In [18]:
threshold = max(dfg[('score', 'count')])/2
threshold = 0
dftmp = dfg[dfg[('score','count')] > threshold]
dftmp.sort_values(by=('score','mean'), ascending=False)

score                                          \
                           count      mean    median       std       min   
shrinkage       components                                                 
0.270526315789  2              8  0.818002  0.830239  0.093608  0.651163   
0.583157894737  4              3  0.794462  0.821429  0.092255  0.691729   
0.374736842105  2              2  0.784188  0.784188  0.053788  0.746154   
0.270526315789  3              6  0.736283  0.695973  0.121917  0.592593   
0.635263157895  4              8  0.711846  0.720506  0.123421  0.511811   
0.531052631579  3              8  0.699045  0.699861  0.130354  0.481481   
0.791578947368  4              3  0.694103  0.692308  0.055022  0.640000   
0.426842105263  2              4  0.693240  0.677976  0.149881  0.547445   
0.166315789474  3              4  0.692644  0.736165  0.133191  0.500000   
0.426842105263  5             24  0.686706  0.681391  0.116309  0.440000   
0.218421052632  4              9  0.680344  0.691729  0.109898  0.523077   
0.478947368421  2              5  0.674558  0.656489  0.066305  0.600000   
0.374736842105  6             41  0.673837  0.678571  0.123731  0.392857   
0.218421052632  3             10  0.673082  0.691655  0.109366  0.500000   
0.635263157895  6             34  0.672664  0.666667  0.127979  0.320000   
0.114210526316  5             17  0.670755  0.649635  0.132206  0.413793   
0.583157894737  6             34  0.668891  0.674435  0.101690  0.384615   
                5             12  0.666805  0.639900  0.094728  0.560000   
0.374736842105  3              3  0.666328  0.640288  0.074164  0.608696   
0.322631578947  3              5  0.666133  0.653846  0.077924  0.604651   
0.478947368421  6             36  0.665032  0.656633  0.105802  0.384615   
0.791578947368  2              6  0.663801  0.709656  0.118149  0.500000   
0.322631578947  6             43  0.662402  0.654867  0.107093  0.392857   
0.531052631579  6             42  0.660036  0.685617  0.095289  0.461538   
0.218421052632  5             21  0.654774  0.641791  0.109237  0.500000   
0.114210526316  3              8  0.651835  0.682747  0.187096  0.434783   
0.270526315789  5             14  0.651019  0.667582  0.127872  0.444444   
0.739473684211  6             24  0.649596  0.639744  0.098584  0.461538   
0.0621052631579 2              8  0.648326  0.592033  0.192394  0.370370   
0.270526315789  6             35  0.647790  0.669231  0.108035  0.375000   
...                          ...       ...       ...       ...       ...   
auto            4             16  0.596489  0.612500  0.142731  0.346154   
0.374736842105  5             11  0.593023  0.567797  0.090832  0.482759   
0.895789473684  4              5  0.592989  0.555556  0.130503  0.459854   
0.0621052631579 4              8  0.591605  0.605897  0.081418  0.481481   
0.947894736842  6             19  0.589250  0.576923  0.060340  0.500000   
                4              4  0.588011  0.592450  0.019917  0.560000   
0.895789473684  5              5  0.582813  0.550388  0.158386  0.423077   
0.166315789474  2              9  0.579215  0.636364  0.123664  0.344828   
auto            2             12  0.578850  0.565396  0.109116  0.458333   
1.0             4              9  0.578020  0.551471  0.125694  0.428571   
0.0621052631579 5             14  0.573552  0.549825  0.108796  0.407407   
                3              9  0.573464  0.580882  0.114872  0.346154   
0.843684210526  3              3  0.573272  0.551724  0.049256  0.538462   
0.687368421053  2              2  0.571070  0.571070  0.008277  0.565217   
0.01            5             18  0.563988  0.563492  0.092080  0.384615   
0.843684210526  5              5  0.561568  0.538462  0.120175  0.407407   
0.947894736842  2              4  0.551741  0.553571  0.075344  0.474820   
0.635263157895  2              5  0.547697  0.529412  0.164711  0.346154   
0.01            6             36  0.544996  0.540970  0.095549  0.384615   

In [23]:
#print(dftmp.sort_values(by=('score','mean'), ascending=False).to_latex())

---
## 2.4 KNN

In [26]:
model = 'knn_ss10_201_600_raw'
model = 'knn_ss4_201_700_fft_None_80_40bins_nopower'

scores = []; ks = []

for resfile in glob.glob('results/'+model+'*.npz'):
    objs = np.load(resfile)
    config = dotdict(objs["config"].reshape(-1)[0])
    results = dotdict(objs["results"].reshape(-1)[0])
    data = results.data
    for comb in combinations(np.arange(10), 2):
        score = data[comb][0]
        scores.append(score[0])
        ks.append(score[1]['knn__n_neighbors'])

df_knn = pd.DataFrame({'score':scores, 'k':ks})

In [27]:
threshold = max(dfg[('score', 'count')])/2
#threshold = 0
dfg = df_knn.groupby(['k']).agg({'score':  ['count', 'mean', 'median', 'std', 'min', 'max']})
dftmp = dfg[dfg[('score', 'count')] > threshold]
dftmp.sort_values(by=('score','mean'), ascending=False)

score                                                  
   count      mean    median       std       min       max
k                                                         
13   254  0.577781  0.572039  0.083906  0.259259  0.821429
14   228  0.571825  0.576091  0.084505  0.260870  0.794118
12   104  0.571661  0.571429  0.087335  0.344828  0.884615
11   186  0.569810  0.566553  0.072728  0.346154  0.750000
10    85  0.566837  0.571429  0.092984  0.370370  0.880000
8     84  0.560006  0.545367  0.106202  0.307692  0.857143
7    147  0.558910  0.560000  0.090985  0.240000  0.777778
9    153  0.553672  0.555556  0.099214  0.241379  0.851852
5    127  0.544978  0.545455  0.080761  0.344828  0.777778
6     72  0.536363  0.539304  0.080111  0.250000  0.714286

In [28]:
#print(dftmp.sort_values(by=('score','mean'), ascending=False).to_latex())